# Yolov4 People Detection
This notebook is a copy of https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial modified to detect and extract people bounding box in order to crop the image to be fed to Deeplabv3 notebook

## Path definitions - change to your own
Generate a txt file with IMAGES_PATH/image_name in each line

In [1]:
import glob
import os

In [2]:
IMAGES_PATH = "/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedImgs"
IMG_LIST = "/content/drive/MyDrive/TFM/yolov4_results/images.txt"
RESULT_JSON = "/content/drive/MyDrive/TFM/yolov4_results/result.json"
TRANSFORM_CSV = "/content/transform.csv"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open(IMG_LIST, 'w') as images_path:
  for image in glob.glob(f'{IMAGES_PATH}/*.jpg'):
    # Check duplicity due to Google Drive duplicity
    if '(1)' in image:
      os.remove(image)
      continue
    images_path.write(f'{image}\n')

## Cloning and Building Darknet
The following cells will clone darknet repository, adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet.

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [6]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

## Download pre-trained YOLOv4 weights
YOLOv4 has been trained already on the coco dataset which has 80 classes that it can predict, including people.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## Define Helper Functions

In [9]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## Runing detections with Darknet/YOLOv4 and save results to JSON File

The object detector can be run using the following command
```bash
!./darknet detector test <path to .data file> <path to config> <path to weights> <path to image>
```

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -thresh 0.7 -ext_output -dont_show -out {RESULT_JSON} < {IMG_LIST}

In [ ]:
!python /content/drive/MyDrive/TFM/yolo2faster_rcnn_format.py -i {RESULT_JSON} -o {TRANSFORM_CSV} -v